In [ ]:
hot_T = pd.DataFrame({'temp':temp_hot}, index=time) rolls_H = hot_T.rolling('300s').std()
#print(rolls) plot_H = rolls_H.plot(title='Hot blackbody rolling standard deviation',fontsize=14, figsize=[16,5], grid=True, legend=False) plot_H.title.set_size(20) plt.savefig('Standard_deviation_hot.png')
cold_T = pd.DataFrame({'temp':temp_cold}, index=time) rolls_C = cold_T.rolling('300s').std()
#print(rolls) plot_C = rolls_C.plot(title='Cold blackbody rolling standard deviation', fontsize=14, figsize=[16,5], grid=True, legend=False) plot_C.title.set_size(20) plt.savefig('Standard_deviation_cold.png')

In [ ]:
rolls_H.reset_index().values[10,1] #reset_index converts it into a Python array

In [ ]:
def func(num):
    temp_hot_slope = []
    temp_cold_slope = []
    for i in range(len(new_temp_hot)-num):
        temp_hot_slope.append((new_temp_hot[i+num]-new_temp_hot[i]))
        temp_cold_slope.append((new_temp_cold[i+num]-new_temp_cold[i]))

    lags = np.arange(-len(temp_hot_slope) + 1, len(temp_hot_slope))
    ccov = np.correlate(temp_hot_slope - np.mean(temp_hot_slope), temp_cold_slope - np.mean(temp_cold_slope), mode='full') #full means all points where the two functions overlap are used. Therefore boundary effects are visible
    ccor = ccov / (len(temp_hot_slope) * np.std(temp_hot_slope) * np.std(temp_cold_slope))
    maxlag = lags[np.argmax(ccor)]
    return(maxlag*2.1/60)

list1 = []

for n in range(3000):
    list1.append(func(n))

plt.scatter(np.arange(3000),list1)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm

def plot_3d_fns_0(start, stop, azimuth=None, elevation=None): #start and stop is roughly the time in seconds
    fig = plt.figure(figsize=(16,9))
    ax = fig.add_subplot(111, projection='3d')
    
    ax.scatter(wns_last, rad_last, temps_last, cmap='inferno')


plot_deltas(0, wns_last[-1])

In [ ]:
def plot_3d_fn(start, stop, azimuth=None, elevation=None): #start and stop is roughly the time in seconds
    fig = plt.figure(figsize=(16,9))
    ax = fig.add_subplot(111, projection='3d')
    
    ax.scatter(wns_first, temps_first, rad_first, cmap='inferno')
    ax.set_xlabel('Wavenumber')
    ax.set_ylabel('Temperature')
    ax.set_zlabel('Radiance')

plot_3d_fn(0, wns_first[-1])